<center>
<h2/>
<h2>INF582
<br>Lab Session 6: Finetuning LLMs</h2> 16 / 02 / 2024<br> Dr. G. Shang<br><br>


<b>Student name:</b> Abdoul Rahim ZEBA<br>

</center>

<hr style="border:10px solid gray"> </hr>
<p style="text-align: justify;">
This handout includes theoretical introductions, <font color='blue'>coding tasks</font> and <font color='red'>questions</font>. Before the deadline, you should submit to Moodle a <B>.ipynb</B> file named <b>Lastname_Firstname.ipynb</b> containing your notebook (with the gaps filled and your answers to the questions). Your answers should be well constructed and well justified. They should not repeat the question or generalities in the handout. When relevant, you are welcome to include figures, equations and tables derived from your own computations, theoretical proofs or qualitative explanations. One submission is required for each student. The deadline for this lab is <b>February 23
, 2024 08:29 AM</b>. No extension will be granted. Late policy is as follows: ]0, 24] hours late → -5 pts; ]24, 48] hours late → -10 pts; > 48 hours late → not graded (zero).
</p>
<hr style="border:5px solid gray"> </hr>

In this lab you will learn how to use HuggingFace transformers - The most used library by researchers and developers  and finetune language models - to finetune a pretrained French language model ($RoBERTa_{small}^{fr}$) on the sentiment analysis dataset CLS_Books where each review is labeled as positive or negative and finetune a variant of BLOOM on a question/answer dataset.

## <b>Preparing the environment and installing libraries, models and data</b>

In this section, we will setup the environment on Google Colab (first cell), download the pretraind model (second cell) and the finetuning dataset (third cell).

In [2]:
!nvidia-smi

'nvidia-smi' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [3]:
!mkdir inf582.lab6 && cd inf582.lab6 && mkdir libs
%cd inf582.lab6/libs
!git clone https://github.com/huggingface/transformers.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install -qqq accelerate datasets evaluate sentencepiece tensorboardX
!pip install -qqq bitsandbytes peft loralib einops
%load_ext tensorboard

c:\Users\HP\Desktop\INF582\inf582.lab6\libs


Cloning into 'transformers'...
Updating files:  25% (1051/4067)
Updating files:  26% (1058/4067)
Updating files:  27% (1099/4067)
Updating files:  28% (1139/4067)
Updating files:  29% (1180/4067)
Updating files:  30% (1221/4067)
Updating files:  31% (1261/4067)
Updating files:  32% (1302/4067)
Updating files:  33% (1343/4067)
Updating files:  34% (1383/4067)
Updating files:  35% (1424/4067)
Updating files:  36% (1465/4067)
Updating files:  37% (1505/4067)
Updating files:  38% (1546/4067)
Updating files:  39% (1587/4067)
Updating files:  40% (1627/4067)
Updating files:  41% (1668/4067)
Updating files:  42% (1709/4067)
Updating files:  43% (1749/4067)
Updating files:  44% (1790/4067)
Updating files:  45% (1831/4067)
Updating files:  46% (1871/4067)
Updating files:  47% (1912/4067)
Updating files:  48% (1953/4067)
Updating files:  49% (1993/4067)
Updating files:  49% (2033/4067)
Updating files:  50% (2034/4067)
Updating files:  51% (2075/4067)
Updating files:  52% (2115/4067)
Updating fil

  Cloning https://github.com/huggingface/transformers.git to c:\users\hp\appdata\local\temp\pip-req-build-gbw22ats
  Resolved https://github.com/huggingface/transformers.git to commit ce4fff0be7f6464d713f7ac3e0bbaafbc6959ae5
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
   ---------------------------------------- 330.1/330.1 kB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 53.0/53.0 kB 2.9 MB/s eta 0:00:00
   ---------------------------------------- 268.7/268.7 kB 5.5 MB/s eta 0:00:00
   ---------------------------------------- 2.2/2.2 MB 15.6 MB/s eta 0:00:00
  Created wheel for tran

  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git 'C:\Users\HP\AppData\Local\Temp\pip-req-build-gbw22ats'
DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.1 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
spyder 4.1.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 4.1.4 requires pyqtwebengine<5.13; python_version >= "3", which is not installed.
sagemaker 2.168.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 7.0.1 which is incompatible.
sagemaker 2.168.0 requires Py


  Attempting uninstall: safetensors
    Found existing installation: safetensors 0.3.1
    Uninstalling safetensors-0.3.1:
      Successfully uninstalled safetensors-0.3.1
  Attempting uninstall: packaging
    Found existing installation: packaging 20.4
    Uninstalling packaging-20.4:
      Successfully uninstalled packaging-20.4
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.15.1
    Uninstalling huggingface-hub-0.15.1:
      Successfully uninstalled huggingface-hub-0.15.1
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.13.3
    Uninstalling tokenizers-0.13.3:
      Successfully uninstalled tokenizers-0.13.3
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sagemaker 2.168.0 requires importlib-metadata<5.0,>=1.4.0, but you have importlib-metadata 7.0.1 which is incompatible.
sagemaker 2.168.0 requires protobuf<4.0,>=3.1, but you have protobuf 4.25.3 which is incompatible.
sagemaker 2.168.0 requires PyYAML==6.0, but you have pyyaml 5.3.1 which is incompatible.
tensorboard 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.
tensorflow 2.10.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 4.25.3 which is incompatible.


In [4]:
!cd .. && mkdir models
%cd ../models

!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/yYQjg9XWekttG5j/download/RoBERTa_small_fr_HuggingFace.zip" -O "model_huggingface.zip"
!unzip model_huggingface.zip
!rm model_huggingface.zip
!rm -rf __MACOSX/

c:\Users\HP\Desktop\INF582\inf582.lab6\models


'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


In [5]:
!cd .. && mkdir data
%cd ../data
!wget -c "https://nuage.lix.polytechnique.fr/index.php/s/EBHqfR776oCE2Nj/download/cls.books.zip" -O "cls.books.zip"
!unzip cls.books.zip
!rm cls.books.zip
!rm -rf __MACOSX/
!mkdir cls.books-json
%cd ..

c:\Users\HP\Desktop\INF582\inf582.lab6\data


'wget' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'unzip' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.
'rm' n'est pas reconnu en tant que commande interne
ou externe, un programme ex�cutable ou un fichier de commandes.


c:\Users\HP\Desktop\INF582\inf582.lab6


# <b>Part 1: Finetuning $RoBERTa_{small}^{fr}$ using HuggingFace's Transfromers</b>
In this section of the lab, we will finetune a HuggingFace checkpoint of our $RoBERTa_{small}^{fr}$ on the CLS_Books dataset.

The model is based on RoBERTa. It has four transformer's encoder layers and pretrained with the MLM (Masked Language Model) task for 10 epochs using two
Nvidia RTX A6000 GPUs on 8M French documents (approximately 39 GB of French raw text) from the dataset mC4. Our dictionary contains 32k tokens obtained by extracting the most 32k frequent French tokens from the dictionary of XLM-R.

<b><h4><font color='red'>
<hr style="border:10px solid red"> </hr>
Question 1: Compute the number of parameters of the model manually.</b><br>
Hints:

*   For details about the architecture check [here](https://github.com/hadi-abdine/fairseq/blob/main/fairseq/models/roberta/model.py#L693-L700).
*   You can omit the biases and the parameters of normalization layers.
*   Don't count the parameters of the language model head.

<hr style="border:10px solid red"> </hr>
</font></h4>

<b><h4><font color='green'>
<hr style="border:10px solid green"> </hr>
Answer 1: </b><br>

Based on the architecture of the model, we have 4 transformer's encoder layers. Each layer has 3 sub-layers: a multi-head self-attention mechanism, a feed-forward network, and a normalization layer. The size of the hidden layer is 512. If we omit the biases and the parameters of normalization layer and don't count the parameters of the language model head, we have the following number of parameters for each layer:

*Multi-head self-attention mechanism:* 
There are total 8 attention heads, and in each head, there are Q, K, V vectors each with dimension 512 × 512.
The total number of parameters for each head is 512 * 512 * 3 * 8 = 786,432.

*Feed-forward network:*
512 * 2048 * 4 = 4,194,304

*Total number of parameters for each layer:*
 12,582,912 + 4,194,304 = 16,777,216

*Total number of parameters for the model:*
16,777,216 * 4 = 67,108,864

<hr style="border:10px solid green"> </hr>
</font></h4>

 We will start by downloading the HuggingFace checkpoint and <b>preparing a json format of the CLS_Books dataset</b> (Which is suitable for HuggingFace's checkpoints finetuning using their run_glue script).

## <b>Converting the CLS_Books dataset to json line files</b>

In order to use the implemented script in the transformers library, you need to convert your data to json line files (for each split: train, valid and test). For instance, each line inside you file will consist of one and one sample only, contaning the review (accessed by the key <i>sentence1</i> and its label, accessed by the key <i>label</i>. Below you can find an example from <i>valid.json</i> file.

Note that these instructions are not valid for all kind of tasks. For other types of tasks (supported in Hugging face) you have to refer to their github for more details.<br>

---------------------------------------------------------------------
<i>
{"sentence1":"Seul ouvrage fran\u00e7ais sur le th\u00e8me Produits Structur\u00e9s \/ fonds \u00e0 formule, il permet de fa\u00e7on p\u00e9dagogique d'appr\u00e9hender parfaitement les m\u00e9canismes financiers utilis\u00e9s. Une r\u00e9f\u00e9rence pour ceux qui veulent comprendre les technicit\u00e9s de base et les raisons de l'engouement des investisseurs sur ces actifs \u00e0 hauteur de plusieurs milliards d'euros.","label":"1"}<br>
{"sentence1":"Livre tr\u00e8s int\u00e9ressant !  mais si comme moi vous cherchez des \"infos\" sur les techniques de sorties et autres \"modes d'emploi\", afin de vivre par vous m\u00eame ce genre d'exp\u00e9rience, c'est pas le bon livre.  \u00e7a ne lui enl\u00e8ve d'ailleurd rien \u00e0 son int\u00earet.","label":"0"}
</i>

---------------------------------------------------------------------



In [6]:
import json

SPLITS=['train', 'test', 'valid']

for split in SPLITS:
    with open('data/cls.books/'+split+'.review', 'r') as f:
        reviews = f.readlines()
    with open('data/cls.books/'+split+'.label', 'r') as f:
        labels = f.readlines()
    with open('data/cls.books-json/'+split+'.json', 'w') as f:
        #fill the gap here to create train.json, valid.json and test.json
        for review, label in zip(reviews, labels):
            data = {'sentence1': review, 'label': label}
            data = json.dumps(data)
            f.write(data+'\n')


FileNotFoundError: [Errno 2] No such file or directory: 'data/cls.books/train.review'

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 1: </b><br>
Finetune the pretrained model on the dataset CLS-Books to perform binary text
classification:

*   Use three different seeds.
*   For each seed, choose the checkpoint with best validation accuracy.
*   Report the average accuracy and its standard deviation on the test set.

<hr style="border:10px solid blue"> </hr>
</font></h4>

## <b>Finetuning $RoBERTa_{small}^{fr}$ using the Transformers Library</b>

In order to finetune the model using HuggingFace, you need to use the <b>run_glue.py</b> Python script located in the transformers library. For more details, refer to <a href="https://github.com/huggingface/transformers/tree/main/examples/pytorch/text-classification" target="_blank">the Huggingface/transformers repository on Github</a>. <br/>

Make sure to use the following hyper-parameters: $\textit{batch size}=8, \textit{max number of epochs}: 5, \textit{optimizer}: Adam, \textit{max learning rate}: 1e-05,  \textit{warm up ratio}: 0.06, \textit{learning rate scheduler}: linear$

In [ ]:
DATA_SET='books'
MODEL='RoBERTa_small_fr_huggingface'
MAX_SENTENCES= 8 # fill me, batch size.
LR=1e-05 #fill me, learning rate
MAX_EPOCH=5 #fill me
NUM_CLASSES=2 #fill me
SEEDS=3
CUDA_VISIBLE_DEVICES=0

In [ ]:
for SEED in range(SEEDS):
  SAVE_DIR= 'checkpoints/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/'+str(SEED)
  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \ 

  )  
#fill me


SyntaxError: invalid syntax (<ipython-input-1-0b1ba5b19938>, line 4)

<b><h4><font color='blue'>
<hr style="border:10px solid blue"> </hr>
Task 2: </b><br>
Finetune a random checkpoint of the model on CLS-Books and compare the result with the one from the previous task.

<hr style="border:10px solid blue"> </hr>
</font></h4>

## <b>Finetuning a non-pretrained $RoBERTa_{small}^{fr}$ using the Transformers Library</b>

In [ ]:
# create a random model with same configuration as RoBERTa Small:
from transformers import AutoModel, AutoConfig, AutoTokenizer
model = AutoModel.from_config(AutoConfig.from_pretrained('models/RoBERTa_small_fr_HuggingFace/'))
tokenizer = AutoTokenizer.from_pretrained('models/RoBERTa_small_fr_HuggingFace/')
tokenizer.save_pretrained('models/RoBERTa_small_fr_HuggingFace_random/')
model.save_pretrained('models/RoBERTa_small_fr_HuggingFace_random/')

for SEED in range(SEEDS):
  SAVE_DIR= 'checkpoints/'+DATA_SET+'/'+MODEL+'_ms'+str(MAX_SENTENCES)+'_lr'+str(LR)+'_me'+str(MAX_EPOCH)+'/random_'+str(SEED)
  !(python libs/transformers/examples/pytorch/text-classification/run_glue.py \

      )#fill me

In [ ]:
%tensorboard --logdir checkpoints --port 6007

# <b>Part 2: Finetuning BLOOM-560m using HuggingFace's Transfromers</b>
In this lab, we will fintune [BLOOM-560m](https://huggingface.co/bigscience/bloom-560m) on a question/answer dataset.

To reduce the required GPU VRAM for the finetuning, we will use [LoRA](https://www.anyscale.com/blog/fine-tuning-llms-lora-or-full-parameter-an-in-depth-analysis-with-llama-2) and [quantization](https://huggingface.co/blog/4bit-transformers-bitsandbytes) techniques.

## <b>Preparing the environment and installing libraries:<b>

In [ ]:
import json
import os
from pprint import pprint

import bitsandbytes as bnb
import pandas as pd
import torch
import torch.nn as nn
import transformers
from datasets import load_dataset

from peft import (
    LoraConfig,
    PeftConfig,
    PeftModel,
    get_peft_model,
    prepare_model_for_kbit_training,
)
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
)

## <b>Loading the model and the tokenizer:</b>
In this section, we will load the BLOOM model while using the BitsAndBytes library for quantization.

In [ ]:
MODEL_NAME = "bigscience/bloom-560m"
# fill the gap
bnb_config = BitsAndBytesConfig.from_pretrained(MODEL_NAME)

model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    device_map="auto",
    trust_remote_code=True,
    quantization_config=bnb_config,
)

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
def print_trainable_parameters(model):

    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        # fill the gap: get the number of trainable parameters: trainable_params
        trainable_params += param.numel() if param.requires_grad else 0
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

## <b>Configuring LoRA:<b>

In [ ]:
config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM",
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

## <b>Test the model before finetuning:<b>

In [ ]:
# fill the gap, prompt of the format: "<human>: Comment je peux créer un compte?  \n <assistant>: ", with an empty response from the assistant
prompt = "<human>:  Comment je peux créer un compte?  \n <assistant>: "
print(prompt)


generation_config = model.generation_config
generation_config.max_new_tokens = 200
generation_config.temperature = 0.7
generation_config.top_p = 0.7
generation_config.num_return_sequences = 1
generation_config.pad_token_id = tokenizer.eos_token_id
generation_config.eos_token_id = tokenizer.eos_token_id

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

## <b>Loading the question/answer dataset from HuggingFace:<b>

In [ ]:
data = load_dataset("OpenLLM-France/Tutoriel", data_files="ecommerce-faq-fr.json")
pd.DataFrame(data["train"])

## <b>Preparing the finetuning data:<b>

In [ ]:
def generate_prompt(data_point):
    # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: response"
    return f"<human>: {data_point['question']}  \n <assistant>: {data_point['response']}"


def generate_and_tokenize_prompt(data_point):
    full_prompt = generate_prompt(data_point)
    tokenized_full_prompt = tokenizer(full_prompt, padding=True, truncation=True)
    return tokenized_full_prompt

data = data["train"].shuffle().map(generate_and_tokenize_prompt)

## <b>Finetuning:<b>

In [ ]:
OUTPUT_DIR = "experiments"

training_args = transformers.TrainingArguments(
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    save_total_limit=3,
    logging_steps=1,
    output_dir=OUTPUT_DIR,
    max_steps=80,
    optim="paged_adamw_8bit",
    lr_scheduler_type="cosine",
    warmup_ratio=0.05,
    report_to="tensorboard",
)

trainer = transformers.Trainer(
    model=model,
    train_dataset=data,
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

model.config.use_cache = False
trainer.train()

In [ ]:
%load_ext tensorboard
%tensorboard --logdir experiments/runs --port 6008

## <b>Test the model after the finetuning:<b>

In [ ]:
%%time
device = "cuda:0"

encoding = tokenizer(prompt, return_tensors="pt").to(device)
with torch.inference_mode():
    outputs = model.generate(
        input_ids=encoding.input_ids,
        attention_mask=encoding.attention_mask,
        generation_config=generation_config,
    )
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
def generate_response(question: str) -> str:
    # fill the gap, transform the data into prompts of the format: "<human>: question?  \n <assistant>: " with an empty response
    prompt = f"<human>: {question}?  \n <assistant>: "
    encoding = tokenizer(prompt, return_tensors="pt").to(device)
    with torch.inference_mode():
        outputs = model.generate(
            input_ids=encoding.input_ids,
            attention_mask=encoding.attention_mask,
            generation_config=generation_config,
        )
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    assistant_start = "<assistant>:"
    response_start = response.find(assistant_start)
    return response[response_start + len(assistant_start) :].strip()

In [ ]:
prompt = "Puis-je retourner un produit s'il s'agit d'un article en liquidation ou en vente finale ?"
print('-', prompt,'\n')
print(generate_response(prompt))

prompt = "Que se passe-t-il lorsque je retourne un article en déstockage ?"
print('\n\n\n-', prompt, '\n')
print(generate_response(prompt))

print('\n\n\n-', prompt, '\n')
prompt = "Comment puis-je savoir quand je recevrai ma commande ?"
print(generate_response(prompt))